# Transfer learning experiments

This notebook includes:
1) Implement a transfer learning from ImageNet > 13 diseases > 1 disease. 
2) Compared performance against ImageNet > 1 disease and ImageNet > MURA > 1 disease 

Results for disease corresponding to index 10.

## Imports & global variables

We choose `idx=10` because from previous experiments we have detected a strong signal and a sufficient amount of positive cases. 

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import sys; sys.path.append("..")
import warnings; warnings.filterwarnings('ignore')

from core import * 
from data_manipulation import Transform, RandomRotation, Flip, RandomCrop, multi_label_2_binary, balance_obs, DataBatches
from utils import save_model, load_model, lr_loss_plot, resize
from architectures import DenseNet121
from train_functions import OptimizerWrapper, TrainingPolicy, FinderPolicy, validate_multilabel, lr_finder, validate_binary, TTA_binary

seed = 42
r_pix = 8
BATCH_SIZE = 16
EPOCHS = 10
TRANSFORMATIONS = [RandomRotation(arc_width=20), Flip(), RandomCrop(r_pix=r_pix)]
PRETRAINED = True
NORMALIZE = True # ImageNet
FREEZE = True
GRADUAL_UNFREEZING = True
DATA = '14diseases'
N_SAMPLES = 1100


BASE_PATH = Path('../..')
PATH = BASE_PATH/'data'
SAVE_RESULTS = './results/'
SAVE_MODELS = './models'
IMG_FOLDER = PATH/'ChestXRay-250'

DISEASE = 'Emphysema'
tgt2idx = {disease: i for i, disease in enumerate([ 'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia',
               'Pneumothorax', 'Consolidation', 'Edema', 'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia'])}

# Downstream task - 1 label (small data)

Balance training and testing.

In [2]:
def train(n_epochs, train_dl, valid_dl, model, max_lr=.01, wd=0, alpha=1./ 3,
          save_path=None, unfreeze_during_loop:tuple=None):
    
    if unfreeze_during_loop:
        total_iter = n_epochs*len(train_dl)
        first_unfreeze = int(total_iter*unfreeze_during_loop[0])
        second_unfreeze = int(total_iter*unfreeze_during_loop[1])

    best_loss = np.inf
    cnt = 0
    
    policy = TrainingPolicy(n_epochs=n_epochs, dl=train_dl, max_lr=max_lr)
    optimizer = OptimizerWrapper(model, policy, wd=wd, alpha=alpha)

    for epoch in tqdm_notebook(range(n_epochs), ):
        model.train()
        agg_div = 0
        agg_loss = 0
        train_dl.set_random_choices()
        for x, y in tqdm_notebook(train_dl, leave=False):

            if unfreeze_during_loop:
                if cnt == first_unfreeze: model.unfreeze(1)
                if cnt == second_unfreeze: model.unfreeze(0)

            out = model(x)
            loss = F.binary_cross_entropy_with_logits(input=out.squeeze(), target=y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            batch = y.shape[0]
            agg_loss += batch*loss.item()
            agg_div += batch
            cnt += 1


        val_loss, measure, _ = validate_binary(model, valid_dl)
        print(f'Ep. {epoch+1} - train loss {agg_loss/agg_div:.4f} -  val loss {val_loss:.4f} AUC {measure:.4f}')

        if save_path and val_loss < best_loss:
            save_model(model, save_path)
            best_loss = val_loss

# Comparison on the Downstream task

In [3]:
train_df = pd.read_csv(PATH/'train_df.csv')
valid_df = pd.read_csv(PATH/"val_df.csv")
test_df = pd.read_csv(PATH/"test_df.csv")

train_df = multi_label_2_binary(train_df, tgt2idx[DISEASE])
train_df = train_df.sample(frac=1, random_state=42)

valid_df = multi_label_2_binary(valid_df, tgt2idx[DISEASE])
valid_df = balance_obs(valid_df, amt=None, rate_positive=.5)

test_df = multi_label_2_binary(test_df, tgt2idx[DISEASE])
test_df = balance_obs(test_df, amt=None, rate_positive=.5)

In [4]:
# len(sample_train_df),len(valid_df),len(test_df)

## Without Mixup

In [5]:
p_positive = .40
N = int(p_positive*100)
sample_train_df = balance_obs(train_df, amt=50, rate_positive=p_positive)

In [6]:

train_dl = DataBatches(df=sample_train_df, transforms=TRANSFORMATIONS, shuffle=True,
                       img_folder_path=IMG_FOLDER, batch_size=BATCH_SIZE, data='Pneumonia',
                       r_pix=r_pix, normalize=NORMALIZE, seed=seed, mixup=False)

valid_dl = DataBatches(df=valid_df, transforms=None, shuffle=False,
                       img_folder_path=IMG_FOLDER, batch_size=BATCH_SIZE, data='Pneumonia',
                       r_pix=r_pix, normalize=NORMALIZE, seed=seed, mixup=False)

test_dl = DataBatches(df=test_df, transforms=TRANSFORMATIONS, shuffle=False, 
                      img_folder_path=IMG_FOLDER, batch_size=BATCH_SIZE, data='Pneumonia',
                      r_pix=r_pix, normalize=NORMALIZE, seed=seed, mixup=False)

### ImageNet

In [7]:
drop_rate = 0
model = DenseNet121(out_size=1, pretrained=True, freeze=True, drop_rate=drop_rate).cuda()
model_p = f'./models/best_{N}_emphysema_imagenet.pth'
epochs = 5
train(epochs, train_dl, valid_dl, model, max_lr=1e-3, save_path=None, 
      unfreeze_during_loop = (.1,.2))
TTA_binary(model, test_dl)

Ep. 1 - train loss 0.7102 -  val loss 0.7086 AUC 0.5189


Ep. 2 - train loss 0.6244 -  val loss 0.9075 AUC 0.7162


Ep. 3 - train loss 0.4204 -  val loss 0.8733 AUC 0.7692


Ep. 4 - train loss 0.2648 -  val loss 0.8215 AUC 0.7762


Ep. 5 - train loss 0.2604 -  val loss 0.7673 AUC 0.7799

TTA loss 0.8395  auc 0.6510  accuracy 0.5252


(0.839544363129008, 0.6509761444574933, 0.52516010978957)

### MURA

In [10]:
drop_rate = 0 # Not good for CNN
model = DenseNet121(out_size=1, pretrained='MURA', freeze=True, drop_rate=drop_rate).cuda()
model_p = f'./models/best_{N}_emphysema_MURA.pth'
epochs = 3
train(epochs, train_dl, valid_dl, model, max_lr=1e-3, save_path=None, 
      unfreeze_during_loop = (.1,.2))
TTA_binary(model, test_dl)

Ep. 1 - train loss 0.7672 -  val loss 0.6887 AUC 0.6637


Ep. 2 - train loss 0.4849 -  val loss 0.6559 AUC 0.7594


Ep. 3 - train loss 0.3898 -  val loss 0.6525 AUC 0.7631

TTA loss 0.6913  auc 0.6391  accuracy 0.5549


(0.6913081686483646, 0.639142543123545, 0.5548947849954254)

### CheXpert

In [21]:
drop_rate = 0 # Not good for CNN
model = DenseNet121(out_size=1, pretrained='chexpert', freeze=True, drop_rate=drop_rate).cuda()
model_p = f'./models/best_{N}_emphysema_chexpert.pth'
epochs = 12
train(epochs, train_dl, valid_dl, model, 
      max_lr=1e-3, save_path=None, 
      unfreeze_during_loop = (.1,.2))
TTA_binary(model, test_dl)

Ep. 1 - train loss 0.7063 -  val loss 0.7446 AUC 0.3547


Ep. 2 - train loss 0.6981 -  val loss 0.7172 AUC 0.4227


Ep. 3 - train loss 0.6796 -  val loss 0.6916 AUC 0.6876


Ep. 4 - train loss 0.6618 -  val loss 0.6979 AUC 0.7233


Ep. 5 - train loss 0.6556 -  val loss 0.7235 AUC 0.7315


...

Ep. 9 - train loss 0.6414 -  val loss 0.6774 AUC 0.7832


Ep. 10 - train loss 0.6153 -  val loss 0.6750 AUC 0.7830


Ep. 11 - train loss 0.6127 -  val loss 0.6727 AUC 0.7842


Ep. 12 - train loss 0.6133 -  val loss 0.6750 AUC 0.7836
TTA loss 0.6931  auc 0.6916  accuracy 0.5005


(0.6930845703892171, 0.6916257411172654, 0.5004574565416285)

### 13 diseases

In [10]:
# drop_rate = 0 # Not good for CNN
# model = DenseNet121(out_size=1, pretrained='13diseases', freeze=True, drop_rate=drop_rate).cuda()
# model_p = f'./models/best_{N}_emphysema_13diseases.pth'
# epochs = 7
# train(epochs, train_dl, valid_dl, model, max_lr=1e-3, save_path=None, 
#       unfreeze_during_loop = (.1,.2))
# TTA_binary(model, test_dl)

Ep. 1 - train loss 0.6420 -  val loss 0.6437 AUC 0.7965


Ep. 2 - train loss 0.6059 -  val loss 0.6200 AUC 0.8123


Ep. 3 - train loss 0.5613 -  val loss 0.6639 AUC 0.8129


Ep. 4 - train loss 0.5432 -  val loss 0.6802 AUC 0.8169


Ep. 5 - train loss 0.4838 -  val loss 0.6329 AUC 0.8198


Ep. 6 - train loss 0.4633 -  val loss 0.6372 AUC 0.8197


Ep. 7 - train loss 0.4737 -  val loss 0.6590 AUC 0.8190

TTA loss 0.6358  auc 0.7860  accuracy 0.7118


(0.6358496082532548, 0.7859706072662347, 0.7118023787740164)

## With Mixup

In [19]:
train_dl = DataBatches(df=sample_train_df, transforms=TRANSFORMATIONS, shuffle=True,
                       img_folder_path=IMG_FOLDER, batch_size=BATCH_SIZE, data='Pneumonia',
                       r_pix=r_pix, normalize=NORMALIZE, seed=seed, mixup=True)

valid_dl = DataBatches(df=valid_df, transforms=None, shuffle=False,
                       img_folder_path=IMG_FOLDER, batch_size=BATCH_SIZE, data='Pneumonia',
                       r_pix=r_pix, normalize=NORMALIZE, seed=seed, mixup=False)

test_dl = DataBatches(df=test_df, transforms=TRANSFORMATIONS, shuffle=False, 
                      img_folder_path=IMG_FOLDER, batch_size=BATCH_SIZE, data='Pneumonia',
                      r_pix=r_pix, normalize=NORMALIZE, seed=seed, mixup=False)

### ImageNet

In [12]:
drop_rate = 0 # Not good for CNN
model = DenseNet121(out_size=1, pretrained=True, freeze=True, drop_rate=drop_rate).cuda()
model_p = f'./models/best_{N}_emphysema_imagenet_mixup.pth'
epochs = 6
train(epochs, train_dl, valid_dl, model, max_lr=1e-3, save_path=None, 
      unfreeze_during_loop = (.1,.2))
TTA_binary(model, test_dl)

Ep. 1 - train loss 0.7032 -  val loss 0.7066 AUC 0.5441


Ep. 2 - train loss 0.6170 -  val loss 0.7853 AUC 0.7066


Ep. 3 - train loss 0.5351 -  val loss 0.9712 AUC 0.7376


Ep. 4 - train loss 0.4395 -  val loss 0.9941 AUC 0.7615


Ep. 5 - train loss 0.4200 -  val loss 0.9740 AUC 0.7645


Ep. 6 - train loss 0.3582 -  val loss 0.7518 AUC 0.7671

TTA loss 0.8453  auc 0.6322  accuracy 0.5668


(0.845337565071465, 0.6321739690905027, 0.5667886550777677)

### MURA

In [ ]:
drop_rate = 0 # Not good for CNN
model = DenseNet121(out_size=1, pretrained='MURA', freeze=True, drop_rate=drop_rate).cuda()
model_p = f'./models/best_{N}_emphysema_MURA_mixup.pth'
epochs = 6
train(epochs, train_dl, valid_dl, model, max_lr=1e-3, save_path=None, 
      unfreeze_during_loop = (.1,.2))
TTA_binary(model, test_dl)

Ep. 1 - train loss 0.7034 -  val loss 0.8257 AUC 0.4079


Ep. 2 - train loss 0.6373 -  val loss 0.8497 AUC 0.6609


Ep. 3 - train loss 0.5488 -  val loss 0.8814 AUC 0.7482


Ep. 4 - train loss 0.4402 -  val loss 0.8903 AUC 0.7575


Ep. 5 - train loss 0.3466 -  val loss 0.7880 AUC 0.7572


Ep. 6 - train loss 0.3647 -  val loss 0.7360 AUC 0.7597

TTA loss 0.8193  auc 0.6067  accuracy 0.5425


(0.819336811671918, 0.6066894962453406, 0.5425434583714547)

### CheXpert

In [ ]:
drop_rate = 0 # Not good for CNN
model = DenseNet121(out_size=1, pretrained='chexpert', freeze=True, drop_rate=drop_rate).cuda()
model_p = f'./models/best_{N}_emphysema_chexpert_mixup.pth'
epochs = 6
train(epochs, train_dl, valid_dl, model, max_lr=5e-4, save_path=None, 
      unfreeze_during_loop = (.1,.2))
TTA_binary(model, test_dl)

Ep. 1 - train loss 0.6980 -  val loss 0.6853 AUC 0.5098


Ep. 2 - train loss 0.6915 -  val loss 0.6661 AUC 0.5829


Ep. 3 - train loss 0.6833 -  val loss 0.6699 AUC 0.5962


Ep. 4 - train loss 0.6783 -  val loss 0.6732 AUC 0.6439


Ep. 5 - train loss 0.6601 -  val loss 0.6743 AUC 0.6289


Ep. 6 - train loss 0.6700 -  val loss 0.6739 AUC 0.6230

TTA loss 0.6917  auc 0.6015  accuracy 0.5000


(0.6917180198518268, 0.6015063838834671, 0.5)

### 13 diseases

In [ ]:
drop_rate = 0 # Not good for CNN
model = DenseNet121(out_size=1, pretrained='13diseases', freeze=True, drop_rate=drop_rate).cuda()
model_p = f'./models/best_{N}_emphysema_13diseases_mixup.pth'
epochs = 5
train(epochs, train_dl, valid_dl, model, max_lr=1e-3, save_path=None, 
      unfreeze_during_loop = (.1,.2))
TTA_binary(model, test_dl)

Ep. 1 - train loss 0.6848 -  val loss 0.6650 AUC 0.6879


Ep. 2 - train loss 0.6252 -  val loss 0.6665 AUC 0.7433


Ep. 3 - train loss 0.6007 -  val loss 0.6995 AUC 0.7669


Ep. 4 - train loss 0.6117 -  val loss 0.7378 AUC 0.7779


Ep. 5 - train loss 0.5737 -  val loss 0.7094 AUC 0.7819



# Results


| Method | test AUC   | test Accuracy (tr = .5) |
|------|------|-------| 
|   ImageNet + Mixup  | 0.611 | 0.552|
|   ImageNet  | 0.625 | 0.624|
|   MURA + Mixup  | 0.659| 0.615|
|   MURA  | 0.674 | 0.606|
|   CheXpert + Mixup  | 0.748| 0.513|
|   CheXpert  | 0.769 | 0.594 |

Mixup seems to don't help in the case of X-rays. 